In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # Visualization
import seaborn as sns #Visualization
plt.rcParams['figure.figsize'] = [8,5]
plt.rcParams['font.size'] =14
plt.rcParams['font.weight']= 'bold'
import scipy 
from scipy import stats 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, accuracy_score, auc
import statsmodels.formula.api as smf
import math
from sklearn.model_selection import GridSearchCV, cross_validate, GroupKFold
import warnings
warnings.filterwarnings('ignore')


def calculate_thresholds(n_records, n_bads, n_goods):
    # Total threshold
    if n_records <= 3000: tot_thresh = 0.1
    elif n_records > 3000 and n_records <= 5000: tot_thresh = 0.05
    elif n_records > 5000 and n_records <= 20000: tot_thresh = 0.03
    elif n_records > 20000 and n_records <= 50000: tot_thresh = 0.01
    else: tot_thresh = 0.005
    # Bad threshold
    if n_bads <= 3000: bad_thresh = 0.10
    elif n_bads > 3000 and n_bads <= 5000: bad_thresh = 0.05
    elif n_bads > 5000 and n_bads <= 20000: bad_thresh = 0.03
    elif n_bads > 20000 and n_bads <= 50000: bad_thresh = 0.01
    else:bad_thresh = 0.005
    # Good threshold
    if n_goods <= 3000: good_thresh = 0.10
    elif n_goods > 3000 and n_goods <= 5000: good_thresh = 0.05
    elif n_goods > 5000 and n_goods <= 20000: good_thresh = 0.03
    elif n_goods > 20000 and n_goods <= 50000: good_thresh = 0.01
    else: good_thresh = 0.005
    return tot_thresh, bad_thresh, good_thresh

def calculate_scorecard(d1, char):
    d2 = d1.groupby([char], dropna=False)
    d3 = pd.DataFrame(d2["X"].min(), columns=["min"])
    d3["# Good"] = d2["Y"].sum()
    d3["# Bad"] = d2["Y"].count() - d3["# Good"]
    d3["% Good"] = round(d3["# Good"] / d3["# Good"].sum() * 100, 1)
    d3["% Bad"] = round(d3["# Bad"] / d3["# Bad"].sum() * 100, 1)
    d3["# Total"] = d2["Y"].count()
    d3["% Total"] = round(d3["# Total"] / d3["# Total"].sum() * 100, 1)
    d3["Information Odds"] = round(d3["% Good"] / d3["% Bad"], 2)
    d3["Bad Rate"] = round(d3["# Bad"] / (d3["# Bad"] + d3["# Good"]) * 100, 2)
    d3["WoE"] = round(np.log(d3["% Good"] / d3["% Bad"]), 2)
    iv = (d3["% Good"] - d3["% Bad"]) * d3["WoE"] / 100
    d4 = d3.sort_index().drop(columns=["min"], axis=1)

    return d4, iv

def featureMonotonicBinning(Y, X, char):
    r = 0
    bad_flag = 0
    n = 20
    while np.abs(r) != 1 and bad_flag == 0:
        d1 = pd.DataFrame({"X": X, "Y": Y})
        d1["Value"], bins = pd.qcut(d1["X"], n, duplicates="drop", retbins=True, precision=3)
        if len(bins) == 2:
            bins = bins.tolist()
            bins.insert(0, float("-inf"))
            bins.append(float("+inf"))
            d1["Value"] = pd.cut(d1["X"], bins=bins, precision=3, include_lowest=True)
        d2 = d1.groupby("Value", as_index=True)
        r,p = stats.spearmanr(d2.mean().X, d2.mean().Y)
        d3, iv = calculate_scorecard(d1, "Value")
        d3.dropna(inplace=True)
        
        if len(d3) < 3:
            bad_flag = 1
        n = n-1
       
    pctThresh, badThresh, goodThresh = calculate_thresholds(d3["# Total"].sum(),d3["# Bad"].sum(),d3["# Good"].sum())
    condition = [(d3["% Total"] < pctThresh*100) | (d3["% Bad"] < badThresh*100) | (d3["% Good"] < goodThresh*100)]
    d3["Not Robust"] = np.select(condition, [1], 0)
    criteria = d3["Not Robust"].sum()
    d3 = d3.reset_index()
    while criteria > 0:
        i = d3[d3["Not Robust"] == 1].index[0]
        #if first row -> merge two first categories
        if i == 0:
            bins = np.delete(bins, 1)
        # if last row -> merge two last categories
        elif i == (len(d3) - 1):
            bins = np.delete(bins, len(d3)-1)
        else:
            # if number of samples greater in former -> merge with latter
            if (d3.at[i-1 , "# Total"] > d3.at[i+1 , "# Total"]):
                bins = np.delete(bins, i+1)
            # if number of samples greater in latter -> merge with former
            else:
                bins = np.delete(bins, i)        
        d1 = pd.DataFrame({"X": X, "Y": Y, "Value": pd.cut(X, bins, precision=3, include_lowest=True)})
        d3, iv = calculate_scorecard(d1, "Value")
        condition = [
            (d3["% Total"] < pctThresh*100) | 
            (d3["% Bad"] < badThresh*100) | 
            (d3["% Good"] < goodThresh*100) ]
        d3["Not Robust"] = np.select(condition, [1], 0)
        criteria = d3["Not Robust"].sum()
        d3 = d3.reset_index()
    d3 = d3.drop(columns=["Not Robust"])
    infValue = round(iv.sum(),3)

    ax1 = d3.plot.bar(x='Value', y='WoE', rot=0)
    ax1.legend("")
    ax1.tick_params(axis='both', which='major', labelsize=9)
    ax1.set_title("Train Sample - "+char+" WoE - Inf. Value: " + str(infValue), fontsize=10)
    ax1.set_xlabel(char, fontsize = 10)
    ax1.set_ylabel("Weight of Evidence", fontsize = 10)
    display(d3)
    return d3, iv, infValue




def featureMonotonicBinning(Y, X, char):
    r = 0
    bad_flag = 0
    n = 20
    while np.abs(r) != 1 and bad_flag == 0:
        d1 = pd.DataFrame({"X": X, "Y": Y})
        d1["Value"], bins = pd.qcut(d1["X"], n, duplicates="drop", retbins=True, precision=3)
        if len(bins) == 2:
            bins = bins.tolist()
            bins.insert(0, float("-inf"))
            bins.append(float("+inf"))
            d1["Value"] = pd.cut(d1["X"], bins=bins, precision=3, include_lowest=True)
        d2 = d1.groupby("Value", as_index=True)
        r,p = stats.spearmanr(d2.mean().X, d2.mean().Y)
        d3, iv = calculate_scorecard(d1, "Value")
        d3.dropna(inplace=True)
        
        if len(d3) < 3:
            bad_flag = 1
        n = n-1
       
    pctThresh, badThresh, goodThresh = calculate_thresholds(d3["# Total"].sum(),d3["# Bad"].sum(),d3["# Good"].sum())
    condition = [(d3["% Total"] < pctThresh*100) | (d3["% Bad"] < badThresh*100) | (d3["% Good"] < goodThresh*100)]
    d3["Not Robust"] = np.select(condition, [1], 0)
    criteria = d3["Not Robust"].sum()
    d3 = d3.reset_index()
    while criteria > 0:
        i = d3[d3["Not Robust"] == 1].index[0]
        #if first row -> merge two first categories
        if i == 0:
            bins = np.delete(bins, 1)
        # if last row -> merge two last categories
        elif i == (len(d3) - 1):
            bins = np.delete(bins, len(d3)-1)
        else:
            # if number of samples greater in former -> merge with latter
            if (d3.at[i-1 , "# Total"] > d3.at[i+1 , "# Total"]):
                bins = np.delete(bins, i+1)
            # if number of samples greater in latter -> merge with former
            else:
                bins = np.delete(bins, i)        
        d1 = pd.DataFrame({"X": X, "Y": Y, "Value": pd.cut(X, bins, precision=3, include_lowest=True)})
        d3, iv = calculate_scorecard(d1, "Value")
        condition = [
            (d3["% Total"] < pctThresh*100) | 
            (d3["% Bad"] < badThresh*100) | 
            (d3["% Good"] < goodThresh*100) ]
        d3["Not Robust"] = np.select(condition, [1], 0)
        criteria = d3["Not Robust"].sum()
        d3 = d3.reset_index()
    d3 = d3.drop(columns=["Not Robust"])
    infValue = round(iv.sum(),3)
    display(d3)
    ax1 = d3.plot.bar(x='Value', y='WoE', rot=0)
    ax1.legend("")
    ax1.tick_params(axis='both', which='major', labelsize=9)
    ax1.set_title("Train Sample - "+char+" WoE - Inf. Value: " + str(infValue), fontsize=10)
    ax1.set_xlabel(char, fontsize = 10)
    ax1.set_ylabel("Weight of Evidence", fontsize = 10)
    return d3, iv, infValue



def fMb(Y, X, char):
    r = 0
    bad_flag = 0
    n = 20
    while np.abs(r) != 1 and bad_flag == 0:
        d1 = pd.DataFrame({"X": X, "Y": Y})
        d1["Value"], bins = pd.qcut(d1["X"], n, duplicates="drop", retbins=True, precision=3)
        if len(bins) == 2:
            bins = bins.tolist()
            bins.insert(0, float("-inf"))
            bins.append(float("+inf"))
            d1["Value"] = pd.cut(d1["X"], bins=bins, precision=3, include_lowest=True)
        d2 = d1.groupby("Value", as_index=True)
        r,p = stats.spearmanr(d2.mean().X, d2.mean().Y)
        d3, iv = calculate_scorecard(d1, "Value")
        d3.dropna(inplace=True)
        
        if len(d3) < 3:
            bad_flag = 1
        n = n-1
       
    pctThresh, badThresh, goodThresh = calculate_thresholds(d3["# Total"].sum(),d3["# Bad"].sum(),d3["# Good"].sum())
    condition = [(d3["% Total"] < pctThresh*100) | (d3["% Bad"] < badThresh*100) | (d3["% Good"] < goodThresh*100)]
    d3["Not Robust"] = np.select(condition, [1], 0)
    criteria = d3["Not Robust"].sum()
    d3 = d3.reset_index()
    while criteria > 0:
        i = d3[d3["Not Robust"] == 1].index[0]
        #if first row -> merge two first categories
        if i == 0:
            bins = np.delete(bins, 1)
        # if last row -> merge two last categories
        elif i == (len(d3) - 1):
            bins = np.delete(bins, len(d3)-1)
        else:
            # if number of samples greater in former -> merge with latter
            if (d3.at[i-1 , "# Total"] > d3.at[i+1 , "# Total"]):
                bins = np.delete(bins, i+1)
            # if number of samples greater in latter -> merge with former
            else:
                bins = np.delete(bins, i)        
        d1 = pd.DataFrame({"X": X, "Y": Y, "Value": pd.cut(X, bins, precision=3, include_lowest=True)})
        d3, iv = calculate_scorecard(d1, "Value")
        condition = [
            (d3["% Total"] < pctThresh*100) | 
            (d3["% Bad"] < badThresh*100) | 
            (d3["% Good"] < goodThresh*100) ]
        d3["Not Robust"] = np.select(condition, [1], 0)
        criteria = d3["Not Robust"].sum()
        d3 = d3.reset_index()
    d3 = d3.drop(columns=["Not Robust"])
    infValue = round(iv.sum(),3)
    return d3, iv, infValue



1. Weight Of Evidence (WOE) LOGISTIC REGRSSION SCORECARD

In [2]:
ticker = 'CL=F'
no_years = 100

end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() -
              datetime.timedelta(days=no_years * 365)).strftime('%Y-%m-%d')

print('Ticker: {}'.format(ticker))
print('Start Date: ', start_date)
print('End Date: ', end_date)

df = get_price(ticker, start_date, end_date)
closed_dates_list = get_closed_dates(df)

df['Year'] = df['Date'].astype(str).str[0:4].astype(int)
df['Month'] = df['Date'].astype(str).str[5:7].astype(int)
df['Day'] = df['Date'].astype(str).str[8:10].astype(int)

df


Ticker: CL=F
Start Date:  1924-06-12
End Date:  2024-05-18


,Date,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
0,2000-08-23,31.950001,32.799999,31.950001,32.049999,32.049999,79385,2000,8,23
1,2000-08-24,31.900000,32.240002,31.400000,31.629999,31.629999,72978,2000,8,24
2,2000-08-25,31.700001,32.099998,31.320000,32.049999,32.049999,44601,2000,8,25
3,2000-08-28,32.040001,32.919998,31.860001,32.869999,32.869999,46770,2000,8,28
4,2000-08-29,32.820000,33.029999,32.560001,32.720001,32.720001,49131,2000,8,29
...,...,...,...,...,...,...,...,...,...,...
5955,2024-05-13,78.180000,79.489998,77.779999,79.120003,79.120003,287701,2024,5,13
5956,2024-05-14,79.230003,79.379997,77.680000,78.019997,78.019997,307410,2024,5,14
5957,2024-05-15,78.440002,78.919998,76.699997,78.629997,78.629997,321267,2024,5,15
5958,2024-05-16,78.839996,79.849998,78.199997,79.230003,79.230003,230277,2024,5,16
